# Phindr3D in python

This notebook acts as a basic GUI/framework for Phindr3D to work in python.

In [1]:
import phindr_functions as phi
import phindr_organoidCSApp as org
import phindr_clustering as clu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

# Preliminary set-up:

In [2]:
param = phi.initParameters()

#change parameters as needed
param.tileX = 10
param.tileY = 10
param.tileZ = 3
param.intensityThresholdTuningFactor = 0.5
param.numVoxelBins = 20
param.numSuperVoxelBins = 15
param.numMegaVoxelBins = 40
param.minQuantileScaling = .5
param.maxQuantileScaling = .5
param.randTrainingSuperVoxel = 10000
param.superVoxelThresholdTuningFactor = 0.1
param.megaVoxelTileX = 5
param.megaVoxelTileY = 5
param.megaVoxelTileZ = 2
param.countBackground = False
param.megaVoxelThresholdTuningFactor = 0.5
param.pixelsPerImage = 200 
param.randTrainingPerTreatment = 1
param.randTrainingFields = 5
param.showImage = 0
param.startZPlane = 1
param.endZPlane = 500
param.numRemoveZStart = 1
param.numRemoveZEnd = 1
param.computeTAS = 0
param.showImage = 0
param.trainingPerColumn = False
param.intensityNormPerTreatment = False
param.treatmentColNameForNormalization = ''
param.trainingColforImageCategories = []
param.superVoxelPerField = param.randTrainingSuperVoxel//param.randTrainingFields
param.lowerbound = [0, 0, 0]
param.upperbound = [1, 1, 1]
param.numChannels = 3 


# File loading:
Load folder containing images of interest

NOTE:
-  image names must follow strucuture: (image identification code) + p(z stack number) + ch(channelnumber) + (whatever) + .tiff 
- images in folder also need to have consistent dimensions (constant number of z slices; constant number of channels)

In [3]:
folder_path =  r'D:\Phindr3D_neuron-sample-data' #r'D:\phindr_neuron_testing_subset' #r'D:\Phindr3D_neuron-sample-data'# #r'D:\phindr_neuron_testing_subset'# #r'path_to_images'#

files, imageIDs = phi.get_files(folder_path)

print(files.shape)

#properly calculate number of channels
param.numChannels = int(files.shape[2])

(80, 9, 3)


# Rescale intensities:
want to rescale to between 1 and 0

In [4]:
allImageID = imageIDs
param = phi.getScalingFactorforImages(files, allImageID, param)

print('Lowerbounds:', param.lowerbound)
print('Upperbounds:', param.upperbound)


Lowerbounds: [107.08 187.8  103.04]
Upperbounds: [ 2690.2  23357.32 10929.12]


# Threshold images:
get threshold value to use from training data


In [5]:
param = phi.getImageThresholValues(files, allImageID, param)
print(param.intensityThreshold)


[0.20719605 0.24947556 0.28546007]


# Cluster pixels:
compute pixel categories 

In [6]:
param = phi.getPixelBinCenters(files, allImageID, param)
print(param.pixelBinCenters) #20 different pixel categories made up of scaled intensities of different channels
print(param.pixelBinCenters.shape)

[[0.06320321 0.08291823 0.96777529]
 [0.03505306 0.29356393 0.02877315]
 [0.73207641 0.26837061 0.01295883]
 [0.03638308 0.73157498 0.03293509]
 [0.26365328 0.08003367 0.01117705]
 [0.04061755 0.07086289 0.35711688]
 [0.03966142 0.55575439 0.02112909]
 [0.40547363 0.08347491 0.00924747]
 [0.96970789 0.14629197 0.01102933]
 [0.02418955 0.27938847 0.28974762]
 [0.23630514 0.31001955 0.01221765]
 [0.05328219 0.10370958 0.60909903]
 [0.34102945 0.20050192 0.87839267]
 [0.03064181 0.96843752 0.02937153]
 [0.31030692 0.17499715 0.34196681]
 [0.0147855  0.41210568 0.69594199]
 [0.58171178 0.10341057 0.01350416]
 [0.0230419  0.56666905 0.28653123]
 [0.42189537 0.24393657 0.02105971]
 [0.03577328 0.40946813 0.03772557]]
(20, 3)


# Define super voxels:
use pixel categories to make some supervoxels! (also cluster super voxels into bins/categories)

In [7]:
param = phi.getSuperVoxelBinCenters(files, allImageID, param)
print(param.supervoxelBincenters) #15 different supervoxel categories made up of frequencies of 20 different pixel categories
print(param.supervoxelBincenters.shape)


c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:1254: RuntimeWarning: invalid value encountered in true_divide
  superVoxelProfile = np.divide(superVoxelProfile, np.array([np.sum(superVoxelProfile, axis=1)]).T) #dont worry about divide by zero errors, they are supposed to happen here!


[[0.12323061 0.00973138 0.05221324 0.14848619 0.03124731 0.0829379
  0.05828102 0.02072022 0.04544342 0.0536404  0.02952649 0.00323918
  0.05973034 0.00985795 0.00349916 0.03517409 0.0197144  0.02791381
  0.18541289 0.        ]
 [0.20488914 0.00664026 0.06672453 0.10084923 0.03099158 0.07222038
  0.04135188 0.02067112 0.04094409 0.04460094 0.0238803  0.00351133
  0.16889781 0.00770886 0.00294591 0.03228592 0.00937179 0.01343074
  0.10808418 0.        ]
 [0.07537109 0.01261736 0.05029182 0.25920377 0.04508649 0.05134369
  0.10303213 0.04393016 0.04158095 0.04012933 0.0411543  0.00536195
  0.04239585 0.01088897 0.00593739 0.06313659 0.01614202 0.03066382
  0.06173231 0.        ]
 [0.15125652 0.01044699 0.15790174 0.13525447 0.02352692 0.08348998
  0.04724084 0.02573054 0.05008657 0.03795895 0.01932462 0.0025989
  0.08947345 0.00547926 0.00368418 0.03611357 0.01382061 0.02092326
  0.08568863 0.        ]
 [0.09766017 0.00839516 0.08984098 0.19284565 0.02502252 0.15482317
  0.06372679 0.017

# Combine super voxels to mega voxels:
use super voxel categories and tileprofile to get megavoxels. also get megavoxel bins/categories

In [8]:
param = phi.getMegaVoxelBinCenters(files, allImageID, param)
print(param.megaVoxelBincenters)
print(param.megaVoxelBincenters.shape)

c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:1254: RuntimeWarning: invalid value encountered in true_divide
  superVoxelProfile = np.divide(superVoxelProfile, np.array([np.sum(superVoxelProfile, axis=1)]).T) #dont worry about divide by zero errors, they are supposed to happen here!


[[ 6.93889390e-18  2.77233040e-01  6.20689655e-03  1.15371727e-01
  -1.38777878e-17  1.29826966e-01  1.93779677e-02  1.51240696e-01
   6.93889390e-18  5.95998297e-03  6.93889390e-18  2.85949176e-01
   8.83354619e-03  0.00000000e+00  0.00000000e+00]
 [ 9.25563540e-02  7.89375443e-03  4.08492200e-02  2.60445987e-02
   9.25022474e-02  5.05578565e-02  1.67186142e-01  0.00000000e+00
   3.60376184e-02  1.05055462e-01  4.60306380e-02  7.43412466e-02
   1.03880819e-01  1.57064042e-01  0.00000000e+00]
 [ 2.34270631e-01  6.59412483e-02  1.23732194e-01  8.24518183e-02
   5.20113002e-02  4.93995547e-02  3.51404151e-02 -6.93889390e-18
   4.75582370e-02  8.98264263e-02  5.60252819e-02  4.30453207e-02
   4.68012162e-02  7.37963561e-02  0.00000000e+00]
 [ 6.93889390e-18  4.84687627e-01  1.11034799e-02  3.09650997e-02
   1.48809524e-03  1.37097409e-01  4.44893841e-02  5.61450278e-02
  -2.08166817e-17  1.25459571e-02  6.93889390e-18  2.05429046e-01
   1.43822073e-02  1.66666667e-03  0.00000000e+00]
 [ 3

# Profile images:


In [9]:
outputFolder = ''
outputFileName ='imagefeatures.csv'

param, resultIM, resultRaw, df = phi.extractImageLevelTextureFeatures(files, allImageID, param, outputFileName=outputFileName, outputDir=outputFolder)
df

c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:1254: RuntimeWarning: invalid value encountered in true_divide
  superVoxelProfile = np.divide(superVoxelProfile, np.array([np.sum(superVoxelProfile, axis=1)]).T) #dont worry about divide by zero errors, they are supposed to happen here!
c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:850: RuntimeWarning: invalid value encountered in true_divide
  megaVoxelProfile = np.divide(megaVoxelProfile, np.array([np.sum(megaVoxelProfile, axis=1)]).T) #hopefully this works, they ask for elementwise, but the arrays seem to have different shapes.
c:\Users\teole\anaconda3\envs\phy479\phindr_functions.py:604: RuntimeWarning: invalid value encountered in true_divide
  imageProfile = imageProfile / np.sum(imageProfile) #normalize the image profile



All done.


,ImageID,numMV,MV1,MV2,MV3,MV4,MV5,MV6,MV7,MV8,...,MV31,MV32,MV33,MV34,MV35,MV36,MV37,MV38,MV39,MV40
0,D:\Phindr3D_neuron-sample-data\r03c19f01,1445.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,D:\Phindr3D_neuron-sample-data\r03c19f02,260.0,0.007692,0.030769,0.026923,0.015385,0.126923,0.000000,0.007692,0.080769,...,0.011538,0.053846,0.057692,0.000000,0.003846,0.007692,0.000000,0.000000,0.023077,0.0
2,D:\Phindr3D_neuron-sample-data\r03c19f03,889.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.067492,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.031496,0.000000,0.000000,0.060742,0.046119,0.000000,0.0
3,D:\Phindr3D_neuron-sample-data\r03c19f04,1001.0,0.000000,0.143856,0.000999,0.000000,0.012987,0.000000,0.000000,0.004995,...,0.019980,0.045954,0.000000,0.000000,0.000000,0.000999,0.000000,0.000000,0.000000,0.0
4,D:\Phindr3D_neuron-sample-data\r03c19f05,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,D:\Phindr3D_neuron-sample-data\r04c22f06,251.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
76,D:\Phindr3D_neuron-sample-data\r04c22f07,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,D:\Phindr3D_neuron-sample-data\r04c22f08,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,D:\Phindr3D_neuron-sample-data\r04c22f09,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import pickle
with open('paramfile.pkl', 'wb') as f:
    pickle.dump(param, f, pickle.HIGHEST_PROTOCOL)


# Cluster images:

# Classify images: